In [1]:
from DataIO import DataIO, City, Service, TrafficType
from SleepInference.SleepDetector import SleepDetector
from datetime import date

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
city = City.DIJON
service = Service.FACEBOOK_MESSENGER

In [ ]:
xar_city = DataIO.load_traffic_data(city=city, traffic_type=TrafficType.B)

In [4]:
sleep_detector = SleepDetector(xar_city=xar_city,city=city, window=10)

In [11]:
sleep = sleep_detector.calculate_sleep_tile_time_day()